# S3W10D1：深度学习数学内功——反向传播（Backpropagation）

## 1 理论知识讲解

**核心概念**：
反向传播（backpropagation）本质上是**链式法则（Chain Rule）**在计算图上的应用。我们把神经网络看作一个复合函数：
$$Y = f_n(...f_2(f_1(X))...)$$
我们要计算Loss对某个权重$W$的梯度 $\frac{\partial L}{\partial W}$，就是将从Output端出啊u你回来的“误差信号”（上游梯度）乘以长纤层的“局部梯度”。

**今日符号约定**（Batch First）：

- 输入 $X$: 形状 $(N, D_{in})$
- 权重 $W$: 形状 $(D_{in}, D_{out})$
- 偏移 $b$: 形状 $(1, D_{out})$
- 线性输出 $Z = XW + b$: 形状 $(N, D_{out})$
- 激活输出 $A = \sigma(Z)$: 形状 $(N, D_{out})$

> **思考题**：在矩阵运算中， 的形状必须和  一样吗？
> *答案：是的。这是检查公式推导是否正确的唯一标准。*

## 2 代码实现 (Numpy 手搓神经网络)

在这一步，我们**不使用 PyTorch 的 Autograd**，而是使用 NumPy 手动实现前向和反向传播。这能强迫你理解矩阵转置的必要性。

In [2]:
import numpy as np

N, D_in, H, D_out = 64, 1000, 100, 10

x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6

print("开始训练...")

for t in range(500):
    # --- Forward Pass (前向传播) ---
    h = x.dot(w1)           # Layer 1 Linear: (N, D_in) * (D_in, H) -> (N, H)
    h_relu = np.maximum(h, 0) # Activation (ReLU)
    y_pred = h_relu.dot(w2) # Layer 2 Linear: (N, H) * (H, D_out) -> (N, D_out)

    # Compute Loss (MSE)
    loss = np.square(y_pred - y).sum()
    if t % 100 == 0:
        print(f"Epoch {t}: Loss = {loss}")

    # --- Backward Pass (反向传播 - 核心难点) ---
    
    # 1.求 Loss 对 y_pred 的梯度
    # dL/d(y_pred) = 2 * (y_pred - y)
    grad_y_pred = 2.0 * (y_pred - y) # Shape: (N, D_out)

    # 2. 求 Layer 2 权重的梯度 (dL/dw2)
    # 链式法则: dL/dw2 = dL/dy_pred * dy_pred/dw2
    # dy_pred/dw2 其实就是上一层的输入 h_relu
    # 维度分析: (H, D_out) = (H, N) * (N, D_out) -> 需要转置 h_relu
    grad_w2 = h_relu.T.dot(grad_y_pred)

    # 3. 求 Layer 1 激活后的梯度 (也就是这一层的误差信号回传)
    grad_h_relu = grad_y_pred.dot(w2.T) # Shape: (N, H)

    # 4. 求 ReLU 的梯度 (大于0导数为1，小于0导数为0)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0 # ReLU 导数逻辑

    # 5. 求 Layer 1 权重的梯度 (dL/dw1)
    grad_w1 = x.T.dot(grad_h) # Shape: (D_in, H)

    # --- Update Weights (SGD) ---
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

print("训练完成。如果你能看懂上面的 .T (转置) 操作，恭喜你入门了。")

开始训练...
Epoch 0: Loss = 25274173.275672376
Epoch 100: Loss = 242.845908972135
Epoch 200: Loss = 0.9359920740023366
Epoch 300: Loss = 0.005408707256518394
Epoch 400: Loss = 3.3754851178167285e-05
训练完成。如果你能看懂上面的 .T (转置) 操作，恭喜你入门了。


## 3 深度理论 (LaTeX 手推练习)

请在 Notebook 中双击此单元格，补充完成以下数学推导。你需要证明为什么代码里要写 `.T`。

**设定**：
$L$ 为标量损失函数，$Z = XW$，其中 $X \in \mathbb{R}^{N \times D}$, $W \in \mathbb{R}^{D \times M}$, $Z \in \mathbb{R}^{N \times M}$。
假设我们已知$\frac{\partial L}{\partial Z}$（记为 $\delta_Z$），其形状为$(N, M)$。

**推导目标**：证明  $\frac{\partial L}{\partial W} = X^T \delta_Z$

**你的推导 (请尝试用 LaTeX 写出)**:

> 提示：考虑矩阵中单个元素 $Z_{ij} = \sum_{k=1}^D X_{ik} W_{kj}$
> 1. 根据链式法则：$\frac{\partial L}{\partial W_{kj}} = \sum_{i=1}^N \frac{\partial L}{\partial Z_{ij}} \frac{\partial Z_{ij}}{\partial W_{kj}}$
> 2. 分析 : ... (此处请补充)
> 3. 最终组合成矩阵形式: ...
> 
> 

## 4 模拟面试问答

**Q1: 在反向传播中，如果我把所有权重初始化为 0 会发生什么？**

> **考察点**: 对称性破坏 (Symmetry Breaking)。
> **参考答案**: 如果所有权重为 0，那么每个神经元接收到的输入是一样的，计算出的梯度也是一样的。结果就是所有神经元都在学完全相同的特征，网络退化成了一个线性模型。必须随机初始化来打破对称性。

**Q2: 为什么代码里的 `grad_w2` 是 `h_relu.T.dot(grad_y_pred)` 而不是 `grad_y_pred.dot(h_relu)`?**

> **考察点**: 对矩阵维度的敏感度。
> **参考答案**: 这是一个矩阵乘法维度匹配的问题。`grad_w2` 的形状必须和 `w2` 一致，即 。
> `h_relu` 是 ，`grad_y_pred` 是 。
> 只有  才能得到正确的维度。从数学意义上讲，这是将 Batch 中所有样本对权重的梯度贡献进行了**累加**。

## 5 工程封装

在实际工程中，我们不需要手写 `.T`。PyTorch 的 `autograd` 会自动构建计算图。

```python
import torch

# 同样的逻辑用 PyTorch 实现
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# requires_grad=True 告诉 PyTorch 追踪这个张量的所有操作
w1 = torch.randn(D_in, H, requires_grad=True)
w2 = torch.randn(H, D_out, requires_grad=True)

learning_rate = 1e-6

# Forward
y_pred = x.mm(w1).clamp(min=0).mm(w2) # .mm 是矩阵乘法, .clamp(min=0) 是 ReLU

# Loss
loss = (y_pred - y).pow(2).sum()

# Backward (工程上的一行代码，背后就是我们刚才手推的过程)
loss.backward()

# 检查梯度
print(f"w1 gradients shape: {w1.grad.shape}") # Should be (1000, 100)
print(f"w2 gradients shape: {w2.grad.shape}") # Should be (100, 10)

```

## 6 今日算法练习

**题目**: LeetCode 104. Maximum Depth of Binary Tree
**思路**:
这是一道递归（DFS）的基础题。

1. **终止条件**: 如果 root 为 None，返回 0。
2. **递归步骤**: 左子树深度 = dfs(root.left)，右子树深度 = dfs(root.right)。
3. **返回值**: max(左, 右) + 1。

```python
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def maxDepth(self, root: Optional[TreeNode]) -> int:
        if not root:
            return 0
        
        left_depth = self.maxDepth(root.left)
        right_depth = self.maxDepth(root.right)
        
        return max(left_depth, right_depth) + 1

```

> **进阶挑战 (Optional)**: 尝试用 `collections.deque` 写一个 BFS (层序遍历) 版本，每遍历一层 depth + 1。这对理解 Week 10 Day 6 的内容很有帮助。

---

## 7 今日任务总结

* [ ] **Math**: 能够独立推导出 Linear Layer 的反向传播公式，并解释清楚矩阵转置的原因。
* [ ] **Code**: 成功运行 NumPy 版本的神经网络，看到 Loss 下降。
* [ ] **Interview**: 能够回答“全零初始化”的问题。
* [ ] **Algo**: AC LeetCode 104。

---

### 导师结语

黄同学，今天的“NumPy 手搓神经网络”是深度学习的成人礼。虽然 PyTorch 很方便，但只有理解了底层这些 `dot` 和 `T`，你才能在将来面对模型不收敛、梯度爆炸时，脑海中浮现出数据流动的图像。

**Action**: 请完成第 3 部分的 LaTeX 推导，如果卡住了，请告诉我，我们一起把那个求和符号展开来看看。